# APEx STAC catalog editor guide

This notebook demonstrates how to add metadata to an APEx STAC catalog.

The STAC API is a standard, so other STAC documentation is also relevant.

## Creating STAC metadata from scratch

In this first part, we show how to start from zero with a very basic example.

In most real-world cases, it is not needed nor recommended to author STAC metadata from scratch directly. Instead, various tools and platforms will generate STAC metadata for you.
The STAC metadata generated here is entirely fictional, and also very much useless. It is not an example of how to create high quality metadata that complies with FAIR principles.

This example mostly serves as a demonstration of the API, which is very simply to use. Most of the more complex examples and tools will avoid this type of direct interaction.


In [3]:

import requests
from owslib.ogcapi.records import Records
from owslib.util import Authentication

In [27]:
from requests_auth import OAuth2ResourceOwnerPasswordCredentials

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r
    


auth = OAuth2ResourceOwnerPasswordCredentials(
        token_url="https://auth.apex.esa.int/realms/apex/protocol/openid-connect/token",
        username="xxx",
        password="yyy",
        client_id="project-a-catalogue",
    )

auth = BearerAuth("""zzz""")

In [28]:

r = Records("https://catalogue.project-a.apex.esa.int",auth=Authentication(auth_delegate=auth))
r

To get some information about our daaset, we can run a simple `gdalinfo`command against a representative asset.

In [19]:
!GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR gdalinfo "/vsicurl/https://eoresults.esa.int/d/APEX_TEST/2020/03/01/europe_aggr-orgc_00-020_mean_100_202003-202210/europe_aggr-orgc_00-020_mean_100_202003-202210.tif"

Driver: GTiff/GeoTIFF
Files: /vsicurl/https://eoresults.esa.int/d/APEX_TEST/2020/03/01/europe_aggr-orgc_00-020_mean_100_202003-202210/europe_aggr-orgc_00-020_mean_100_202003-202210.tif
Size is 40888, 41712
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Referen

In [20]:
from datetime import datetime
import pystac

spatial_extent = pystac.SpatialExtent(bboxes=[[4,51,5,52]])
collection_interval = sorted([datetime(2020,1,1), datetime(2022,1,1)])
temporal_extent = pystac.TemporalExtent(intervals=[collection_interval])

description_markdown = """
SOC content in the 0-20 cm top soil expressed in g kg-1 for 100 m resolution pixels

Applications:

- First globally consistent and contiguous complete gridded soil property map of Europe
- Indicator of soil health, as per Mission Area Soil Health and Food
- By 2030, at least 75% of soils in each EU Member State should be in healthy condition or show a
significant improvement towards meeting accepted thresholds of indicators, to support ecosystem
services

*Reliability*: More than 83 % of the cross-validation points fall within the 70% prediction interval for the bare soil model.
For the vegetated area model 94 % of the points fall within the 90 % prediction interval.
"""

item_assets = {
    "SOC": {
       "type": "image/tiff; application=geotiff; profile=cloud-optimized",
        "title": "SOC",
        "description": "SOC content in the 0-20 cm top soil expressed in g kg-1",
        "data_type": "uint16",
        "nodata": 65535,
        "unit": "g kg-1",
        "roles": [
            "data"
        ]
    }
}

collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)
collection = pystac.Collection(id='esa-worldsoils',
                               description=description_markdown,
                               extent=collection_extent,
                               extra_fields=dict(item_assets=item_assets),
                               license='CC-BY-SA-4.0')

collection.summaries.add("proj:code",["EPSG:3035"])
collection.summaries.add("gsd",[100])
collection.summaries.add("bands",[{
    "title": "SOC",
    "gsd": 100
}])


collection

<Collection id=esa-worldsoils>

In [29]:
coll_dict = collection.to_dict()

default_auth = {
    "_auth": {
        "read": ["anonymous"],
        "write": ["stac-openeo-admin", "stac-openeo-editor"]
    }
}

coll_dict.update(default_auth)

response = requests.post("https://catalogue.project-a.apex.esa.int/collections", auth=auth,json=coll_dict)
response

<Response [403]>

In [21]:
import shapely

item_bbox = [4, 51, 4.5, 51.5]
collection_item = pystac.Item(id='my_unique_item',
                              geometry=shapely.geometry.mapping(shapely.box(*item_bbox)),
                              bbox=item_bbox,
                              datetime=datetime(2024,10,2),
                              collection=collection.id,
                              properties={})

collection_item.common_metadata.gsd = 10
collection_item.common_metadata.platform = 'Sentinel-2'
collection_item.common_metadata.instruments = ['MSI']

asset = pystac.Asset(href="https://myhost.com/my_cog.tif", 
                      media_type=pystac.MediaType.GEOTIFF, roles=["data"])
collection_item.add_asset("the_asset_key", asset)
collection_item

<Item id=my_unique_item>

In [38]:
r.collection_item_create(collection.id, collection_item.to_dict())

True

## Cleaning up

When the demo is done, we can simply clean up by deleting our (useless) test collection.

In [26]:
requests.delete("https://stac-openeo-dev.vgt.vito.be/collections/" + collection.id, auth=auth)

<Response [204]>